---
# Data Science Hackathon
_You are currently looking at the solution of the Data Science Hackathon by PwC Eur._

---

## Impact of Recessions on Housing Prices in Univeristy vs Not Univeristy Towns


**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.


In [27]:
import re
import os
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

In [28]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

In [29]:
# os.getcwd() #check your cwd
# os.chdir('...')   #adjust your cwd

In [30]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [7]:
def isState(line: str):
    return line.endswith('[edit]')

def cleanState(line: str):
    return line.replace('[edit]', '')

def cleanCity(line: str):
    return re.sub(r' \(.+$', '', line)

In [33]:
def get_list_of_university_towns():
    stateTownsList = []

    with open("university_towns.txt", encoding="utf-8") as file:
        currentState = None

        for line in file:
            line = line.strip()
            if isState(line):
                currentState = cleanState(line)
            else:
                stateTownsList.append((currentState, cleanCity(line)))

    return pd.DataFrame(stateTownsList, columns=["State", "RegionName"])

In [9]:
def yearMoToYearQuarter(yearMo: str):
    yearMo = yearMo.split('-')
    month = int(yearMo[1])
    quarter = int((month - 1) / 3) + 1
    return yearMo[0] + 'q' + str(quarter)

In [10]:
def convert_housing_data_to_quarters():
    return pd.read_csv('City_Zhvi_AllHomes.csv') \
            .replace({'State': states}) \
            .set_index(['State', 'RegionName']) \
            .iloc[:, 49:] \
            .groupby(yearMoToYearQuarter, axis=1) \
            .mean()

In [11]:
def readGdp():
    gdp = pd.read_excel('gdplev.xls', sheetname=0, skiprows=8, header=None).iloc[:, [4, 6]]
    gdp.columns = ['year-quarter', 'gdp']
    startIndex = gdp.loc[(gdp['year-quarter'] == '2000q1'), :].index[0]
    gdp = gdp.iloc[startIndex:, :]
    gdp['diff'] = gdp['gdp'] - gdp['gdp'].shift(1)
    return gdp.set_index('year-quarter')

In [18]:
def get_recession_start():
    gdp = readGdp()
    return gdp[(gdp['diff'] < 0) & (gdp['diff'].shift(-1) < 0)].index[0]

In [13]:
def get_recession_end():
    gdp = readGdp()
    recessionStart = get_recession_start()
    return gdp.loc[recessionStart:, :].where((gdp['diff'] > 0) & (gdp['diff'].shift(1) > 0)).dropna().index[0]

In [92]:
def get_recession_bottom():
    gdp = readGdp()
    recessionStart = get_recession_start()
    recessionEnd = get_recession_end()
    return gdp.loc[recessionStart:recessionEnd, 'gdp'].idxmin()

In [93]:
def get_recession_start_before():
    gdp = readGdp()
    return gdp[(gdp['diff'].shift(-1) < 0) & (gdp['diff'].shift(-2) < 0)].index[0]


In [94]:
def house_data_with_universities():
    beforeRecessionColumn = get_recession_start_before()
    recessionBottomColumn = get_recession_bottom()
    houseData = convert_housing_data_to_quarters() \
        .loc[:, [beforeRecessionColumn, recessionBottomColumn]] \
        .reset_index() \
        .dropna()
    houseData['price_ratio'] = houseData[beforeRecessionColumn] / houseData[recessionBottomColumn]
    universityTowns = get_list_of_university_towns()
    universityTowns['isUniversityTown'] = True
    houseDataWithUniversities = pd.merge(houseData, universityTowns, how='left', on=['State', 'RegionName'])
    houseDataWithUniversities['isUniversityTown'].fillna(False, inplace=True)
    return houseDataWithUniversities


In [95]:
def run_ttest():
    houseDataWithUniversities = house_data_with_universities()
    withUni = houseDataWithUniversities[houseDataWithUniversities['isUniversityTown']].loc[:, 'price_ratio'].dropna()
    withoutUni = houseDataWithUniversities[~houseDataWithUniversities['isUniversityTown']].loc[:, 'price_ratio'].dropna()
    withUniMean = withUni.mean()
    withoutUniMean = withoutUni.mean()
    alpha = 0.01
    stats = ttest_ind(withUni, withoutUni)
    p = stats[1]
    return (p < alpha, p, 'university town' if withUniMean < withoutUniMean else 'non-university town')
run_ttest()

(True, 0.0027240637047614541, 'university town')